In [ ]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import scipy.stats as stats
import random
import glob
import os
import re

import utilities as ut
import modularised_utils as mut
from load_results import load_results, list_available_results


from matplotlib.animation import FuncAnimation
from IPython.display import HTML

sns.set_theme(style="whitegrid")
seed = 42
np.random.seed(seed)

In [2]:
label_map_empirical = {
                        'DIROCA (eps_0.107_delta_0.035)': 'DiRoCA_star',
                        'DIROCA (eps_1.0_delta_1.0)': 'DIROCA_1',
                        'DIROCA (eps_2.0_delta_2.0)': 'DIROCA_2',
                        'DIROCA (eps_4.0_delta_4.0)': 'DIROCA_4',
                        'DIROCA (eps_8.0_delta_8.0)': 'DIROCA_8',
                        'GradCA': 'GradCA',
                        'BARYCA': 'BARYCA',
                        'Abs-LiNGAM (Perfect)': 'Abslin_p',
                        'Abs-LiNGAM (Noisy)': 'Abslin_n'
                    }
label_map_gaussian = {
                        'DIROCA (eps_delta_0.111)': 'DiRoCA_star',
                        'DIROCA (eps_delta_1)': 'DIROCA_1',
                        'DIROCA (eps_delta_2)': 'DIROCA_2',
                        'DIROCA (eps_delta_4)': 'DIROCA_4',
                        'DIROCA (eps_delta_8)': 'DIROCA_8',
                        'GradCA': 'GradCA',
                        'BARYCA': 'BARYCA'
                    }

print_label_map  = {
                        'DiRoCA_star':  r'DiRoCA$_{\epsilon_\ell^*, \epsilon_h^*}$',
                        'DIROCA_1':     r'DiRoCA$_{1,1}$',
                        'DIROCA_2':     r'DiRoCA$_{2,2}$',
                        'DIROCA_4':     r'DiRoCA$_{4,4}$',
                        'DIROCA_8':     r'DiRoCA$_{8,8}$',
                        'GradCA':       r'GRAD$_{(\tau, \omega)}$',
                        'BARYCA':       r'BARY$_{(\tau, \omega)}$',
                        'Abslin_p':     r'AbsLin$_{\text{p}}$', 
                        'Abslin_n':     r'AbsLin$_{\text{n}}$'
                    }

### Fix σ, vary α

In [ ]:
plt.rcParams.update({
    "text.usetex": False,                 
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman", "CMU Serif", "DejaVu Serif"],
    "mathtext.fontset": "cm",             
    "mathtext.rm": "serif"
})

# Methods and display labels
methods_to_plot = [
    'DiRoCA_star', 'DIROCA_1', 'DIROCA_2', 'DIROCA_4', 'DIROCA_8', 'GradCA', 'BARYCA', 'Abslin_p', 'Abslin_n'
]
display_names = [print_label_map[m] for m in methods_to_plot]

palette_colors = sns.color_palette('colorblind', n_colors=len(display_names))
color_map = dict(zip(display_names, palette_colors))
color_map = {
    r'DiRoCA$_{\epsilon_\ell^*, \epsilon_h^*}$': '#1f77b4', 
    r'DiRoCA$_{1,1}$': 'gold',
    r'DiRoCA$_{2,2}$': 'darkorange',
    r'DiRoCA$_{4,4}$': 'lightskyblue',
    r'DiRoCA$_{8,8}$': 'violet',
    r'GRAD$_{(\tau, \omega)}$': '#2ca02c',                  
    r'BARY$_{(\tau, \omega)}$': '#d62728',                     
    r'AbsLin$_{\text{p}}$': '#9467bd',                       
    r'AbsLin$_{\text{n}}$': '#8c564b'                       
}
# Fixed noise levels per dataset
noise_level_to_plot = {
    'slc': 5.0,
    'lilucas': 10.0
}

fig, axes = plt.subplots(2, 2, figsize=(18, 12), sharey=False)
fig.subplots_adjust(hspace=0.4)
panels = [
    ((0, 0), 'slc',     'gaussian',  "Gaussian"),
    ((0, 1), 'slc',     'empirical', "Empirical"),
    ((1, 0), 'lilucas', 'gaussian',  "Gaussian"),
    ((1, 1), 'lilucas', 'empirical', "Empirical"),
]

combined_handles = {}  # collect handles from all subplots (for a single legend)
present_display_names_global = set()
for (r, c), experiment, noise_type, title_str in panels:
    ax = axes[r, c]

    # Load per-panel data
    all_data = ut.load_all_data(experiment)
    df = load_results(
        experiment=experiment,
        evaluation_type=noise_type,
        shift_type='additive',
        distribution='gaussian', #'student-t', #'exponential',
        alpha_steps=10,
        alpha_min=0.0,
        alpha_max=1.0,
        noise_steps=20,
        noise_min=0.0,
        noise_max=noise_level_to_plot[experiment],
        trials=20,
        zero_mean=True
    )

    if df is None:
        print(f"No data found for {noise_type} in {experiment}")
        # still style the empty axes
        ax.set_title(title_str, fontsize=30)
        ax.set_xlabel(r'$\alpha$', fontsize=34)
        ax.set_ylabel('Abstraction Error' if c == 0 else '', fontsize=32)
        ax.tick_params(axis='both', labelsize=18)
        ax.grid(True, linestyle='--', alpha=0.7)
        continue

    # Replace method names
    if noise_type == 'empirical':
        df['method'] = df['method'].replace(label_map_empirical)
    else:
        df['method'] = df['method'].replace(label_map_gaussian)

    # Filter subset
    df = df[
        (df['noise_scale'] == noise_level_to_plot[experiment]) &
        (df['method'].isin(methods_to_plot))
    ].copy()

    if df.empty:
        print(f"No data for {noise_type} in {experiment}")
        ax.set_title(title_str, fontsize=30)
        ax.set_xlabel(r'$\alpha$', fontsize=34)
        ax.set_ylabel('Abstraction Error' if c == 0 else '', fontsize=32)
        ax.tick_params(axis='both', labelsize=18)
        ax.grid(True, linestyle='--', alpha=0.7)
        continue

    df['display_name'] = df['method'].map(print_label_map)

    sns.lineplot(
        data=df,
        x='alpha',
        y='error',
        hue='display_name',
        hue_order=display_names,
        palette=color_map,
        marker='o',
        linewidth=2.5,
        markersize=8,
        errorbar='sd',
        ax=ax,
        legend=False
    )
    present_display_names_global.update(df['display_name'].unique())

    ax.set_title(title_str, fontsize=30)
    ax.set_xlabel(r'$\alpha$', fontsize=34)
    ax.set_ylabel('Abstraction Error' if c == 0 else '', fontsize=32)
    ax.tick_params(axis='both', labelsize=18)
    ax.grid(True, linestyle='--', alpha=0.7)

    handles, labels = ax.get_legend_handles_labels()
    for h, l in zip(handles, labels):
        if l and (l not in combined_handles):
            combined_handles[l] = h

present_display_names_ordered = [lbl for lbl in display_names if lbl in present_display_names_global]

if present_display_names_ordered:
    ordered_handles = [
        plt.Line2D([], [], linestyle='-', linewidth=6,
                   label=lbl, color=color_map.get(lbl, '#000000'))
        for lbl in present_display_names_ordered
    ]
    fig.legend(
        ordered_handles,
        present_display_names_ordered,
        loc='lower center',
        ncol=min(6, len(present_display_names_ordered)),
        fontsize=20,
        frameon=False
    )
else:
    print("No legend entries found across panels; skipping legend.")


fig.tight_layout(rect=[0, 0.08, 1, 0.97])
plt.savefig("plots/abstraction_vs_alpha_SLC_LiLuCas_2x2.png", dpi=300, bbox_inches='tight')
plt.show()


### Fix α, vary σ

In [ ]:
fixed_alpha_to_plot = {'slc': 1.0, 'lilucas': 1.0}
noise_level_to_plot = {'slc': 5.0, 'lilucas': 10.0} 

fig, axes = plt.subplots(2, 2, figsize=(18, 12), sharey=False)
fig.subplots_adjust(hspace=0.4)

panels = [
    ((0, 0), 'slc',     'gaussian',  "Gaussian"),
    ((0, 1), 'slc',     'empirical', "Empirical"),
    ((1, 0), 'lilucas', 'gaussian',  "Gaussian"),
    ((1, 1), 'lilucas', 'empirical', "Empirical"),
]

present_display_names_global = set()

for (r, c), experiment, noise_type, title_str in panels:
    ax = axes[r, c]

    # Load results for this panel
    all_data = ut.load_all_data(experiment)
    df = load_results(
        experiment=experiment,
        evaluation_type=noise_type,
        shift_type='additive',
        distribution='gaussian', #'student-t', #'exponential',
        alpha_steps=10,
        alpha_min=0.0,
        alpha_max=1.0,
        noise_steps=20,
        noise_min=0.0,
        noise_max=noise_level_to_plot[experiment],
        trials=20,
        zero_mean=True
    )

    if df is None:
        print(f"No data found for {noise_type} in {experiment}")
        ax.set_title(title_str, fontsize=30)
        ax.set_xlabel(r'$\tilde{\sigma}$', fontsize=34)
        ax.set_ylabel('Abstraction Error' if c == 0 else '', fontsize=32)
        ax.tick_params(axis='both', labelsize=18)
        ax.grid(True, linestyle='--', alpha=0.7)
        continue

    if noise_type == 'empirical':
        df['method'] = df['method'].replace(label_map_empirical)
    else:
        df['method'] = df['method'].replace(label_map_gaussian)

    df = df[
        (np.abs(df['alpha'] - fixed_alpha_to_plot[experiment]) < 1e-4) &
        (df['method'].isin(methods_to_plot))
    ].copy()

    if df.empty:
        print(f"No data for {noise_type} in {experiment} at alpha={fixed_alpha_to_plot[experiment]}")
        ax.set_title(title_str, fontsize=30)
        ax.set_xlabel(r'$\tilde{\sigma}$', fontsize=34)
        ax.set_ylabel('Abstraction Error' if c == 0 else '', fontsize=32)
        ax.tick_params(axis='both', labelsize=18)
        ax.grid(True, linestyle='--', alpha=0.7)
        continue

    df['display_name'] = df['method'].map(print_label_map)

    sns.lineplot(
        data=df,
        x='noise_scale',
        y='error',
        hue='display_name',
        hue_order=display_names,
        palette=color_map,
        marker='o',
        linewidth=2.5,
        markersize=8,
        errorbar='sd',
        ax=ax,
        legend=False
    )
    present_display_names_global.update(df['display_name'].unique())

    ax.set_title(title_str, fontsize=30)
    ax.set_xlabel(r'$\tilde{\sigma}$', fontsize=34)
    ax.set_ylabel('Abstraction Error' if c == 0 else '', fontsize=32)
    ax.tick_params(axis='both', labelsize=18)
    ax.grid(True, linestyle='--', alpha=0.7)

present_display_names_ordered = [lbl for lbl in display_names if lbl in present_display_names_global]

if present_display_names_ordered:
    ordered_handles = [
        plt.Line2D([], [], linestyle='-', linewidth=6,
                   label=lbl, color=color_map.get(lbl, '#000000'))
        for lbl in present_display_names_ordered
    ]
    fig.legend(
        ordered_handles,
        present_display_names_ordered,
        loc='lower center',
        ncol=min(6, len(present_display_names_ordered)),
        fontsize=20,
        frameon=False
    )
else:
    print("No legend entries found across panels; skipping legend.")

fig.tight_layout(rect=[0, 0.08, 1, 0.98])
plt.savefig("plots/abstraction_vs_noise_SLC_LiLuCas_alpha_fixed_2x2.png", dpi=300, bbox_inches='tight')
plt.show()


### Work on a particular setting/example

In [103]:
experiment      = 'slc'
evaluation_type = 'gaussian'

path = f"data/{experiment}/results"
saved_folds = joblib.load(f"data/{experiment}/cv_folds.pkl")

all_data      = ut.load_all_data(experiment)

Dll_samples   = all_data['LLmodel']['data']
Dhl_samples   = all_data['HLmodel']['data']
I_ll_relevant = all_data['LLmodel']['intervention_set']
omega         = all_data['abstraction_data']['omega']
ll_var_names  = list(all_data['LLmodel']['graph'].nodes())
hl_var_names  = list(all_data['HLmodel']['graph'].nodes())

Data loaded for 'slc'.


In [ ]:
final_results_df = load_results(
                                experiment=experiment,
                                evaluation_type=evaluation_type,
                                shift_type='additive',
                                distribution='gaussian',
                                alpha_steps=10,
                                alpha_min=0.0,
                                alpha_max=1.0,
                                noise_steps=20,
                                noise_min=0.0,
                                noise_max=5.0 if experiment == 'slc' else 10.0,
                                trials=20,
                                zero_mean=True
                            )


### 0-shift (α=0)

In [ ]:
df_clean             = final_results_df[final_results_df['alpha']==0.0]
summary_stats        = df_clean.groupby(['method'])['error'].agg(['mean', 'std', 'count'])
summary_stats['sem'] = summary_stats['std']# / np.sqrt(summary_stats['count'])

print("--- Final Summary (Mean Error ± SEM) ---")

print("\n" + "="*65)
print(f"{'Method/Run':<45} | {'Mean ± Std'}")
print("="*65)

for method_name, row in summary_stats.iterrows():
    mean_val = row['mean']
    sem_val = row['sem']
    
    print(f"{method_name:<45} | {mean_val:>7.4f} ± {sem_val:.4f}")

print("="*65)

### ρ-shift (α=1, σ>0)

In [ ]:
alpha_point = 1.0
noise_level_point = 5 if experiment == 'slc' else 10.0

df_point            = final_results_df[(final_results_df['alpha']==alpha_point) & (final_results_df['noise_scale']==noise_level_point)]
summary_stats        = df_point.groupby(['method'])['error'].agg(['mean', 'std', 'count'])
summary_stats['sem'] = summary_stats['std'] / np.sqrt(summary_stats['count'])


print("\n" + "="*65)
print(f"{'Method/Run':<45} | {'Mean ± Std'}")
print("="*65)

for method_name, row in summary_stats.iterrows():
    mean_val = row['mean']
    sem_val = row['sem']
    
    print(f"{method_name:<45} | {mean_val:>7.4f} ± {sem_val:.4f}")

print("="*65)

### GiF generation

In [ ]:
plt.rcParams.update({
    "text.usetex": False,                 
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman", "CMU Serif", "DejaVu Serif"],
    "mathtext.fontset": "cm",             # CM-style math
    "mathtext.rm": "serif"
})

methods_to_plot = ['DiRoCA_star', 'DIROCA_2', 'GradCA', 'BARYCA', 'Abslin_p', 'Abslin_n']
display_names   = [print_label_map[m] for m in methods_to_plot]

color_map = {
    r'DiRoCA$_{\epsilon_\ell^*, \epsilon_h^*}$': '#1f77b4', 
    r'DiRoCA$_{2,2}$': 'darkorange',
    r'GRAD$_{(\tau, \omega)}$': '#2ca02c',
    r'GRAD$_{\text{bary}}$': '#d62728',
    r'AbsLin$_{\text{p}}$': '#9467bd',
    r'AbsLin$_{\text{n}}$': '#8c564b'
}


alpha_values = sorted(final_results_df['alpha'].unique())
noise_levels = sorted(final_results_df['noise_scale'].unique())

if evaluation_type == 'empirical':
    final_results_df['method'] = final_results_df['method'].replace(label_map_empirical)
else:
    final_results_df['method'] = final_results_df['method'].replace(label_map_gaussian)

# Filter to chosen methods
df_for_plotting = final_results_df[final_results_df['method'].isin(methods_to_plot)].copy()
df_for_plotting['display_name'] = df_for_plotting['method'].map(print_label_map)

y_max = df_for_plotting['error'].max()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

def update_alpha(alpha):
    ax.clear()
    df_subset = df_for_plotting[np.isclose(df_for_plotting['alpha'], alpha)]

    sns.lineplot(
        data=df_subset,
        x='noise_scale',
        y='error',
        hue='display_name',
        hue_order=display_names,
        palette=color_map,
        marker='o',
        linewidth=2.5,
        markersize=8,
        errorbar='sd',
        ax=ax
    )

    ax.set_title(rf'$\alpha = {alpha:.2f}$', fontsize=36)
    ax.set_xlabel(r'$\tilde{\sigma}$', fontsize=32)
    ax.set_ylabel('Abstraction Error', fontsize=32)
    ax.tick_params(axis='both', labelsize=16)
    ax.legend(title='Method', fontsize=14, title_fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_ylim(0, y_max * 1.1)


print("Creating animation for evolving alpha...")
ani_alpha = FuncAnimation(fig, update_alpha, frames=alpha_values, blit=False, repeat=False)

# Save the GIF
ani_alpha.save('plots/robustness_vs_noise_evolving_alpha.gif', writer='pillow', fps=2)
plt.close()
# HTML(ani_alpha.to_jshtml())

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

def update_noise_scale(scale):
    """This function is called for each frame to draw the plot for a specific noise scale."""
    ax.clear()
    
    df_subset = df_for_plotting[np.isclose(df_for_plotting['noise_scale'], scale)]
    
    sns.lineplot(
        data=df_subset,
        x='alpha',
        y='error',
        hue='display_name',
        hue_order=display_names,
        palette=color_map,
        marker='o',
        linewidth=2.5,
        markersize=8,
        errorbar='sd',
        ax=ax
    )
    
    ax.set_title(rf'$\tilde{{\sigma}} = {scale:.2f}$', fontsize=36)
    ax.set_xlabel(r'$\alpha$', fontsize=32)
    ax.set_ylabel('Abstraction Error', fontsize=32)
    ax.tick_params(axis='both', labelsize=16)
    ax.legend(title='Method', fontsize=14, title_fontsize=14)
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.set_ylim(0, y_max * 1.1)


print("Creating animation for evolving noise scale...")
ani_noise = FuncAnimation(fig, update_noise_scale, frames=noise_levels, blit=False, repeat=False)

ani_noise.save('plots/robustness_vs_alpha_evolving_noise.gif', writer='pillow', fps=2)
plt.close()
# HTML(ani_noise.to_jshtml())